### Задание 2. Работа с выбросами

В выбранных числовых столбцах найти выбросы, выдвинуть гипотезы об их причинах и проинтерпретировать результаты. Принять и обосновать решение о дальнейшей работе с ними.

In [1]:
import pandas
from HorsesService import HorsesService

In [2]:
horsesService = HorsesService()
horse_data = horsesService.fetch_processed_hourse_data_frame()

In [3]:
def get_outliers_table(
        table: pandas.DataFrame,
        column: str
) -> pandas.DataFrame:
    quantile_1 = table[column].quantile(.15)
    quantile_3 = table[column].quantile(.85)
    print(quantile_1, quantile_3)
    between_table = table[table[column].between(quantile_1, quantile_3)]
    outliers_table = pandas.concat([table, between_table]).drop_duplicates(keep=False)
    return outliers_table

##### Выбросы - rectal_temperature

In [4]:
def get_small_table(hourse_table: pandas.DataFrame) -> pandas.DataFrame:
    table = hourse_table[['rectal_temperature', 'outcome']].sort_values('rectal_temperature')
    table['outcome'] = table['outcome'].apply(lambda row: 'lived' if row == '1' else 'died or euthanized')
    return table

rectal_temperature_table = get_small_table(
    hourse_table=get_outliers_table(
        table=horse_data,
        column='rectal_temperature'
    )
).dropna()
rectal_temperature_table

37.5 38.8


,rectal_temperature,outcome
44,35.4,died or euthanized
141,36.0,died or euthanized
238,36.1,died or euthanized
80,36.4,died or euthanized
118,36.5,lived
...,...,...
20,39.9,lived
281,40.0,died or euthanized
91,40.3,died or euthanized
54,40.3,died or euthanized


Можно проследить связь этих данных с другими данными. При высоких и низких температурах повышенная смертность.  Предпологаю что ошибок в этих данных нет.

##### Выбросы - Pulse

In [5]:
pulse_table = get_outliers_table(
    table=horse_data,
    column='pulse'
)

pulse_table['pulse'].sort_values().dropna()


44.0 100.0


232     30.0
130     30.0
276     36.0
242     36.0
203     36.0
       ...  
41     150.0
275    150.0
55     160.0
3      164.0
255    184.0
Name: pulse, Length: 74, dtype: float64

Не могу сказать, пульс выше 100 это аномальное отклонение или нет. Учитывая что данных выброса много (74 из 300), предполагаю что эти данные в большинстве своем релевантны.

##### Выбросы - respiratory rate

In [6]:
get_outliers_table(
    table=horse_data,
    column='respiratory_rate'
)['respiratory_rate'].sort_values().dropna()

16.0 42.0


190     8.0
203     9.0
140     9.0
162    10.0
89     10.0
       ... 
244    88.0
186    90.0
269    90.0
106    96.0
39     96.0
Name: respiratory_rate, Length: 67, dtype: float64

Не могу сказать, частота дыхания выше 42 это аномальное отклонение или нет. Чтобы принимать решение оставлять эту выборку или нет, надо глубже погрузиться в медицинский показатели.

##### Выбросы - packed cell volume

In [7]:
get_outliers_table(
    table=horse_data,
    column='packed_cell_volume'
)['packed_cell_volume'].sort_values().dropna()

36.0 57.0


190    23.0
56     24.0
65     26.0
287    28.0
189    30.0
       ... 
35     73.0
49     73.0
4      74.0
144    75.0
151    75.0
Name: packed_cell_volume, Length: 73, dtype: float64

Аномальных отклонений не увидел.

##### Выбросы - total protein

In [8]:
get_outliers_table(
    table=horse_data,
    column='total_protein'
)['total_protein'].sort_values().dropna()

6.2 65.0


255     3.3
210     4.0
142     4.5
84      4.5
247     4.6
       ... 
14     81.0
63     82.0
1      85.0
90     86.0
208    89.0
Name: total_protein, Length: 77, dtype: float64

Вся верхняя грань сильно выбивается (> 60), когда норма 6-7.5, предполагаю что данные занесли в других показателях,  не в gms/dL а в g/l. Проблему решит все показатели больше 50 разделить на 10.


##### Выбросы - capillary refill time

In [9]:
get_outliers_table(
    table=horse_data,
    column='capillary_refill_time'
)['capillary_refill_time'].sort_values().dropna()

1.0 2.0


78    3.0
85    3.0
Name: capillary_refill_time, dtype: float64

Возможные показатели 1 или 2, это не релевантные данные

##### Выбросы - age

In [10]:
get_outliers_table(
    table=horse_data,
    column='age'
)['age'].sort_values().dropna()

1.0 1.0


3      9
262    9
255    9
252    9
244    9
229    9
227    9
212    9
191    9
135    9
109    9
103    9
82     9
75     9
74     9
55     9
41     9
39     9
23     9
16     9
13     9
9      9
275    9
287    9
Name: age, dtype: int64

Возможные показатели 1 или 2, это не релевантные данные

##### Выбросы - peripheral pulse

In [11]:
get_outliers_table(
    table=horse_data,
    column='peripheral_pulse'
)['peripheral_pulse'].sort_values().dropna()

1.0 3.0


8      4.0
43     4.0
110    4.0
174    4.0
220    4.0
222    4.0
250    4.0
282    4.0
Name: peripheral_pulse, dtype: float64

Данные корректны